Sample Notebook showing generating embeddings from the S2 encoder

In [22]:
import torch
import xarray as xr

from pixelverse.download.sentinel2 import get_s2_times_series
from pixelverse.models.registry import create_model

In [23]:
SAMPLE_AOI_BBOX = (34.30401965358922, 0.483473285913126, 34.3178177763533, 0.497271408677202)

In [24]:
# download S2 time series data

dset_s2_aoi = get_s2_times_series(bbox=SAMPLE_AOI_BBOX, year=2021)

In [25]:
dset_s2_aoi

<xarray.Dataset> Size: 23MB
Dimensions:      (time: 12, y: 153, x: 154)
Coordinates:
  * time         (time) datetime64[ns] 96B 2021-01-31T08:09:59.930000 ... 202...
  * y            (y) float64 1kB 5.498e+04 5.496e+04 ... 5.346e+04 5.346e+04
  * x            (x) float64 1kB 6.451e+05 6.451e+05 ... 6.466e+05 6.466e+05
    spatial_ref  int32 4B 32636
Data variables:
    blue         (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    green        (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    red          (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    rededge1     (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    rededge2     (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    rededge3     (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    nir          (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    nir08        (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    swir16       (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>
    swir22       (time, y, x) float64 2MB dask.array<chunksize=(1, 153, 154), meta=np.ndarray>

In [26]:
dset_s2_aoi["doy"] = dset_s2_aoi.time.dt.dayofyear  # add day of year as a variable

In [27]:
# Load the S2 encoder model

model = create_model("tessera_s2_encoder", pretrained=True)
model[0].eval()

# Define S2 bands in correct order
s2_bands = [
    "blue",
    "green",
    "red",
    "rededge1",
    "rededge2",
    "rededge3",
    "nir",
    "nir08",
    "swir16",
    "swir22",
]

In [28]:
# Note MVP function, only recommended for small testing areas at this time


def generate_embeddings(s2_dset: xr.Dataset, model_name: str = "tessera_s2_encoder") -> xr.Dataset:
    """
    Generate embeddings for a given Sentinel-2 dataset using the specified model.

    Args:
        model_name (str): Name of the model to use for generating embeddings.
        s2_dataset (rioxarray.DataArray): Sentinel-2 dataset.

    Returns:
        torch.Tensor: Generated embeddings.
    """
    model = create_model(model_name, pretrained=True)
    model[0].eval()

    # Stack spatial dimensions into pixels
    s2_stacked = s2_dset[s2_bands].to_array(dim="band").stack(pixel=["y", "x"])

    # Prepare tensors
    # s2_stacked.values shape: (band, time, pixel) -> transpose to (pixel, time, band)
    s2_tensor = torch.from_numpy(s2_stacked.values.transpose(2, 1, 0))  # (pixel, time, band)

    # DOY tensor: expand to match number of pixels
    doy_tensor = (
        torch.from_numpy(dset_s2_aoi.doy.values).unsqueeze(0).expand(s2_tensor.shape[0], -1)
    )  # (pixel, time)

    # Concatenate bands and DOY along last dimension
    x = torch.cat([s2_tensor, doy_tensor.unsqueeze(-1)], dim=-1)  # (pixel, time, 11)

    # Run through model
    with torch.no_grad():
        embeddings = model[0](x.float())

    # Convert back to xarray with spatial structure
    dset_embeddings = (
        xr.DataArray(
            embeddings.numpy(), dims=["pixel", "feature"], coords={"pixel": s2_stacked.pixel}
        )
        .unstack("pixel")
        .to_dataset(name="embedding")
    )

    # Add back spatial info
    dset_embeddings.rio.write_crs(s2_dset.rio.crs, inplace=True)

    return dset_embeddings

In [29]:
sample_embeddings = generate_embeddings(dset_s2_aoi, model_name="tessera_s2_encoder")

In [30]:
sample_embeddings

<xarray.Dataset> Size: 48MB
Dimensions:      (y: 153, x: 154, feature: 512)
Coordinates:
  * y            (y) float64 1kB 5.498e+04 5.496e+04 ... 5.346e+04 5.346e+04
  * x            (x) float64 1kB 6.451e+05 6.451e+05 ... 6.466e+05 6.466e+05
    spatial_ref  int64 8B 0
Dimensions without coordinates: feature
Data variables:
    embedding    (feature, y, x) float32 48MB 1.459 1.466 1.466 ... 0.3054 0.262

In [31]:
# write to COG

sample_embeddings["embedding"].rio.to_raster(
    "./sample_s2_embeddings_cog.tif",
    driver="COG",
    compress="DEFLATE",
)

In [32]:
# write to zarr
sample_embeddings.to_zarr(
    "./sample_embeddings.zarr",
    mode="w",
)